In [ ]:
import pandas as pd 
import numpy as np 
import folium 



# Load the data
file_path = r'C:\Users\aggel\Downloads\Interview_Programmers_OptiSolio.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')
coordinates = df.iloc[:-2, [1, 2]].values #Coordinates stored in a Numpy array
depot= [35.324662, 25.133215]
demands1= df.iloc[:-2,3].values #Loads of day 31/08
demands2= df.iloc[:-2,4].values #Loads of day 07/09


# Create a map centered around the depot
m = folium.Map(location=depot, tiles='OpenStreetMap', zoom_start=14)
for coord in coordinates:
    folium.Marker(location=coord).add_to(m)

folium.Marker(location=depot, icon=folium.Icon(color="red")).add_to(m)

m



In [ ]:
import openrouteservice as ors
import folium
import pandas as pd


#DAY 31/08


# Initialize ORS Client
client = ors.Client(key='5b3ce3597851110001cf6248d35649aedb9e43ee9f045ce4e63f2b95')  
depot = [35.324662, 25.133215]  

#Vehicle Object
vehicles = [
    ors.optimization.Vehicle(
        id=1,
        start=list(reversed(depot)),  #Convert it to (Long, Lat)
        capacity=[50]                # Capacity of the vehicle
    )
]

#Summing up the data
deliveries_data = pd.DataFrame({
    'Lat': coordinates[:, 0],  
    'Lon': coordinates[:, 1],  
    'Loads1': demands1,        
    'Loads2': demands2         
})

#Jobs object for the API
deliveries = []
for delivery in deliveries_data.itertuples():
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[delivery.Lon, delivery.Lat],
            amount=[delivery.Loads1]
        )
    )

# Perform the optimization request
result = client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True
)


# Extract the route from the result
route = result['routes'][0]

# Decode the polyline (route geometry) to get coordinates
decoded = ors.convert.decode_polyline(route['geometry'])

# Add the route to the existing map (m)
folium.GeoJson(
    name='Day 1 Route',
    data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                     "geometry": decoded,
                                                     "properties": {"color": "green"}
                                                     }]},
    style_function=lambda x: {"color": "green"}
).add_to(m)

m

In [12]:
import openrouteservice as ors
import folium
import pandas as pd

#DAY 07/09



#Vehicle Object
vehicles = [
    ors.optimization.Vehicle(
        id=1,
        start=list(reversed(depot)),  #Convert it to (Long, Lat)
        capacity=[50]                # Capacity of the vehicle
    )
]

#Delivery Jobs for the API
deliveries2 = []
for delivery in deliveries_data.itertuples():
    deliveries2.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[delivery.Lon, delivery.Lat],
            amount=[delivery.Loads2]
        )
    )

# Perform the optimization request
result = client.optimization(
    jobs=deliveries2,
    vehicles=vehicles,
    geometry=True
)

# Add the route to the map 
route = result['routes'][0]
decoded = ors.convert.decode_polyline(route['geometry'])

# Add the decoded route to the map
folium.GeoJson(
    name='Day 2 Route',
    data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                     "geometry": decoded,
                                                     "properties": {"color": "green"}
                                                     }]},
    style_function=lambda x: {"color": "green"}
).add_to(m)

m
